In [83]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from prepare import prepare_data
from math import sqrt

In [42]:
#Hyperparameters
embedding_dim = 64
max_token = 8
d_key = 16

In [2]:
def train_dev_split(x,train,dev):
    N = x.shape[0]
    Ntr = int(N * train)
    Ndev = int(N*dev)
    ind = torch.randperm(Ntr)
    data_tr = x[ind][:Ntr]
    data_dev = x[ind][Ntr:Ntr+Ndev]
    return data_tr,data_dev

In [20]:
device = "cuda" if torch.cuda.device_count() else "cpu"
full_data = prepare_data("input.txt")
data, vocab, encode, decode = torch.tensor(full_data["encoded_data"], device=device), full_data["vocab"], full_data["encode"], full_data["decode"]
vocab_size = len(vocab)
B = data.shape[0] // max_token
ind = torch.randperm(n= data.shape[0] - max_token)[:B]
ranges = ind.view(B,1) + torch.arange(max_token)
data = data[ranges]
data_tr,data_dev = train_dev_split(data,0.9,0.1)

In [21]:
overfitting_data = data[:10]
overfitting_data

tensor([[58, 43, 52, 42,  1, 58, 53,  1],
        [47, 57,  1, 50, 39, 52, 42,  1],
        [42,  1, 41, 39, 52, 53, 54, 47],
        [47, 60, 43,  6,  1, 58, 47, 50],
        [ 1, 53, 59, 58,  1, 53, 44,  1],
        [51, 39, 52, 63,  1, 51, 43, 52],
        [59, 43, 56, 53, 56, 10,  0, 20],
        [46, 39, 52, 49, 44, 59, 50,  1],
        [40, 59, 58,  1, 54, 50, 43, 39],
        [39, 47, 56,  1, 61, 47, 58, 46]])

In [48]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size, embedding_dim)
        self.pos_table = nn.Embedding(max_token,embedding_dim)
    def forward(self,x):
        return self.embedding_table(x) + self.pos_table(torch.arange(max_token))

In [84]:
class Head(nn.Module):
    def __init__(self):
        super().__init__()
        self.keyM = nn.Linear(embedding_dim, d_key)
        self.queryM = nn.Linear(embedding_dim, d_key)
        self.valueM = nn.Linear(embedding_dim, d_key)
    def forward(self,x):
        Q = self.queryM(x)
        K = self.keyM(x)
        V = self.valueM(x)
        scores = (Q @ K.permute(0,2,1)) / sqrt(d_key)
        #Masking for attention
        inp = torch.ones(max_token, max_token)  
        mask = torch.tril(inp).bool()
        scores = scores.masked_fill(~mask, float('-inf'))
        #Attention
        attn = torch.softmax(scores, dim=-1)
        return attn @ V
        

In [82]:
emb = Transformer()
embedded = emb(overfitting_data)
single_head = Head()
current = single_head(embedded)
current

TypeError: sqrt(): argument 'input' (position 1) must be Tensor, not int